In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import password
import re

# Convert CSV to Pandas Dataframe

In [2]:
csv_Apple = "AppleStore.csv"
csv_GPlay = "googleplaystore.csv"
appleDF = pd.read_csv(csv_Apple)
googleDF = pd.read_csv(csv_GPlay)

In [3]:
# Check files imported correctly
# googleDF.head()
# appleDF.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design,"June 20, 2018",1.1,4.4 and up


# File clean-up with selected columns

In [5]:
# Drop columns for apple store
cleaned_apple = appleDF[["track_name","price","user_rating","rating_count_tot","prime_genre"]]
cleaned_apple_df = cleaned_apple.rename(columns={"track_name":"app_name","price":"price","user_rating":"user_rating","rating_count_tot":"number_of_reviews","prime_genre":"genre"})
cleaned_apple_df.head()

,app_name,price,user_rating,number_of_reviews,genre
0,PAC-MAN Premium,3.99,4.0,21292,Games
1,Evernote - stay organized,0.00,4.0,161065,Productivity
2,"WeatherBug - Local Weather, Radar, Maps, Alerts",0.00,3.5,188583,Weather
3,"eBay: Best App to Buy, Sell, Save! Online Shop...",0.00,4.0,262241,Shopping
4,Bible,0.00,4.5,985920,Reference


In [6]:
# Drop columns for google store
cleaned_google = googleDF[["App","Category","Rating","Reviews","Price"]]
cleaned_google_df = cleaned_google.rename(columns = {"App":"app_name","Price":"price","Rating":"user_rating","Reviews":"num_of_reviews","Category":"genre"})
cleaned_google_df.head()

,app_name,genre,user_rating,num_of_reviews,price
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,0


In [7]:
# Remove special characters from DB
for index, record in cleaned_google_df.iterrows():
    y = record['app_name']
    if len(y) > 3:
        #print(y)
        x = re.sub(r'[~|!|@|#|$|%|^|&|*|(|)|"|;|<|>|/|?]',r'',y)
        cleaned_google_df.loc[index, 'app_name'] = x

    else:
        cleaned_google_df.loc[index, 'app_name'] = "NA"

In [8]:
# Remove special characters from DB
for index, record in cleaned_apple_df.iterrows():
    y = record['app_name']
    if len(y) > 3:
        #print(y)
        x = re.sub(r'[~|!|@|#|$|%|^|&|*|(|)|"|;|<|>|/|?]',r'',y)
        cleaned_apple_df.loc[index, 'app_name'] = x

    else:
        cleaned_apple_df.loc[index, 'app_name'] = "NA"

In [9]:
for index, record in cleaned_google_df.iterrows():
       y = record['price']
       if len(y) > 2:
           #print(y)
           x = re.sub(r'[~|!|@|#|$|%|^|&|*|(|)|"|;|<|>|/|?]',r'',y)
           cleaned_google_df.loc[index, 'price'] = x

       else :
            cleaned_google_df.loc[index, 'price'] = 0

In [10]:
for index, record in cleaned_apple_df.iterrows():
       y = str(record['price'])
       if len(y) > 2:
           #print(y)
           x = re.sub(r'[~|!|@|#|$|%|^|&|*|(|)|"|;|<|>|/|?]',r'',y)
           cleaned_apple_df.loc[index, 'price'] = x

       else :
            cleaned_apple_df.loc[index, 'price'] = 0

In [11]:
counter = 0
for index, row in cleaned_google_df.iterrows():
    try:
        location1 = row["genre"].upper()
           #print(row["Category"])
           #print(f"counter value is : {counter}")
        if  location1 == "GAME" :
                   location1 = "GAMES"
        elif location1 == "AUTO_AND_VEHICLES":
                   location1 = "REFERENCE"
        elif location1 == "FAMILY" :
                   location1 = "REFERENCE"
        elif location1 == "PARENTING" :
                location1 = "REFERENCE"
        elif location1 == "TOOLS" :
               location1 = "UTILITIES"
        elif location1 == "PERSONALIZATION" :
                location1 = "UTILITIES"


        elif location1 == "SOCIAL":
                location1 = "SOCIAL_NETWORKING"
        elif location1 == "DATING" :
                location1 = "SOCIAL_NETWORKING"
        elif location1 == "EVENTS" :
                location1 = "SOCIAL_NETWORKING"
        elif location1 == "PHOTOGRAPHY" :
                location1 = "PHOTOS_AND_VIDEOS"
        elif location1 == "VIDEO_PLAYERS" :
                location1 = "PHOTOS_AND_VIDEOS"
        elif location1 == "MAPS_AND_NAVIGATION" :
                location1 = "NAVIGATION"

        elif location1 == "HOUSE_AND_HOME":
                location1 = "CATALOG"
        elif location1 == "1.9" :
                location1 = "CATALOG"
        elif location1 == "BEAUTY" :
                location1 = "CATALOG"
        elif location1 == "ART_AND_DESIGN" :
                location1 = "CATALOG"
        elif location1 == "COMICS" :
                location1 = "CATALOG"
        else :
              location1 = location1

               #print(f"value to be inserted : {location1}")
        cleaned_google_df.loc[index, 'genre'] = location1
        counter = counter+1

        if counter > 100000 :
            break


    except (KeyError, IndexError):
            print("Missing field/some exception - so skipping those")
print("Task Completed....")

Task Completed....


In [12]:
counter = 0
for index, row in cleaned_apple_df.iterrows():

       try:

           location1 = row["genre"].upper()
           #print(row["prime_genre"])
           #print(f"counter value is : {counter}")

           if  location1 == "HEALTH & FITNESS" :
               location1 = "HEALTH_AND_FITNESS"
           elif location1 == "PHOTO & VIDEO":
               location1 = "PHOTOS_AND_VIDIOS"
           elif location1 == "FOOD & DRINK" :
               location1 = "FOOD_AND_DRINK"
           elif location1 == "BOOK" :
               location1 = "BOOKS"
           else :
               location1 = location1

           #print(f"value to be inserted : {location1}")
           cleaned_apple_df.loc[index, 'genre'] = location1
           counter = counter+1

           if counter > 100000 :
                break


       except (KeyError, IndexError):
           print("Missing field/some exception - so skipping those")
print("Task Completed....")

Task Completed....


# Connect to MySQL

In [17]:
rds_connection_string = f'root:{password}@127.0.0.1:3306/etl_project'
engine = create_engine(f'mysql+pymysql://{rds_connection_string}') 

In [18]:
# Verify tables
engine.table_names()

['apple_stats', 'google_stats']

# Use pandas to load dataframe to MySQL

In [19]:
cleaned_google_df.to_sql(name='google_stats', con=engine, if_exists='append', index=False)

InternalError: (pymysql.err.InternalError) (1366, "Incorrect string value: '\\xE1\\xBB\\x87n V...' for column 'app_name' at row 310") [SQL: 'INSERT INTO google_stats (app_name, genre, user_rating, num_of_reviews, price) VALUES (%(app_name)s, %(genre)s, %(user_rating)s, %(num_of_reviews)s, %(price)s)'] [parameters: ({'app_name': 'Photo Editor & Candy Camera & Grid & ScrapBook', 'genre': 'CATALOG', 'user_rating': 4.1, 'num_of_reviews': '159', 'price': 0}, {'app_name': 'Coloring book moana', 'genre': 'CATALOG', 'user_rating': 3.9, 'num_of_reviews': '967', 'price': 0}, {'app_name': 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'genre': 'CATALOG', 'user_rating': 4.7, 'num_of_reviews': '87510', 'price': 0}, {'app_name': 'Sketch - Draw & Paint', 'genre': 'CATALOG', 'user_rating': 4.5, 'num_of_reviews': '215644', 'price': 0}, {'app_name': 'Pixel Draw - Number Art Coloring Book', 'genre': 'CATALOG', 'user_rating': 4.3, 'num_of_reviews': '967', 'price': 0}, {'app_name': 'Paper flowers in

In [ ]:
cleaned_apple_df.to_sql(name='apple_stats', con=engine, if_exists='append', index=False)

In [ ]:
# Confirm data was added
pd.read_sql_query('select * from apple_stats', con=engine).head()